In [1]:
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
from tqdm import tqdm
import pandas as pd
import base as bs
import os as os

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [8]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower().strip()
if " " in empresa:
    empresa_ = empresa.replace(" ", "%20")
else:
    empresa_ = empresa
num_paginas = int(input("Digite la empresa a extraer: "))

In [9]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.portafolio.co/buscar?q={empresa_}')
driver.maximize_window()
WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.XPATH,
                                      './/button[@class="align-right secondary slidedown-button"]')))\
                                          .click()

In [10]:
# Articulos a extraer
url_princ = f'https://www.portafolio.co/buscar?q={empresa_}&page='
titulares = []
for i in tqdm(range(1, num_paginas+1)):
    if i != 1:
        aux = str(i)
        url_a_buscar = url_princ+aux
        driver.get(url_a_buscar)
    articulos = [driver.find_element(By.XPATH, './/div[contains(@class, "listing-item first ")]')] + driver.find_elements(By.XPATH, './/div[contains(@class, "listing-item  ")]')
    
    for art in tqdm(articulos):
        url = art.find_element(By.XPATH, './/h3[@class="listing-title"]//a').get_attribute('href')
        if not(bs.existedb(url, "database")):
            titulo = articulos[0].find_element(By.XPATH, './/h3[@class="listing-title"]//a').text
            fechaP = bs.obtener_fecha_port(art)
            tema = art.find_element(By.XPATH, './/div[@class="listing-category"]').text
            resumen = art.find_element(By.XPATH, './/div[@class="listing-epigraph"]').text
            imagen = bs.obtener_imagen_port(art)
            titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                            'Titulo': titulo,
                            'Fecha Publicacion': fechaP,
                            'URL': url,
                            'Tema': tema,
                            'Resumen': resumen,
                            'Imagen': imagen,
                            'Empresa': empresa,
                            'Fuente': 'Portafolio'
                            })

100%|██████████| 2/2 [00:13<00:00,  6.87s/it]


In [11]:
for tit in tqdm(titulares):
    driver.get(tit["URL"])
    driver.implicitly_wait(10)
    tit['Autor'], tit['Contenido'], tit['RelNewsUrls'] = bs.obtener_autor_contenido_relsnews(driver)

100%|██████████| 20/20 [04:00<00:00, 12.04s/it]


In [13]:
df = pd.DataFrame(titulares)
bs.writeData("database", df)

,Fecha Extraccion,Titulo,Fecha Publicacion,URL,Tema,Resumen,Imagen,Empresa,Fuente,Autor,Contenido,RelNewsUrls
0,2023-03-28 11:03:07,Planta de biomasa para Ecopetrol será el prime...,2023-03-27,https://www.portafolio.co/negocios/empresas/pl...,EMPRESAS,La inversión requerida para su construcción su...,https://www.portafolio.co/files/listing_second...,ecopetrol,Portafolio,DANIELA MORALES SOLER,"En Villanueva, Casanare, se está construyendo ...",https://www.portafolio.co/economia/finanzas/la...
1,2023-03-28 11:03:08,Planta de biomasa para Ecopetrol será el prime...,2023-03-27,https://www.portafolio.co/economia/infraestruc...,INFRAESTRUCTURA,"El vehículo, que fue presentado oficialmente e...",https://www.portafolio.co/files/listing_second...,ecopetrol,Portafolio,PORTAFOLIO,"Ecopetrol, la Alcaldía de Bogotá, Fenoge, Tran...",https://www.portafolio.co/economia/gobierno/mi...
2,2023-03-28 11:03:08,Planta de biomasa para Ecopetrol será el prime...,2023-03-25,https://www.portafolio.co/negocios/empresas/em...,EMPRESAS,"En 11 misivas, la petrolera le pidió múltiples...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",ecopetrol,Portafolio,LINA QUIROGA RUBIO,El diario EL TIEMPO conoció 11 cartas a través...,https://www.portafolio.co/negocios/empresas/ec...
3,2023-03-28 11:03:08,Planta de biomasa para Ecopetrol será el prime...,2023-03-25,https://www.portafolio.co/negocios/empresas/la...,EMPRESAS,"El embajador de Colombia en Venezuela, Armando...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",ecopetrol,Portafolio,DIANA K,Las intenciones de comprar la petroquímica ven...,https://www.portafolio.co/negocios/empresas/ec...
4,2023-03-28 11:03:09,Planta de biomasa para Ecopetrol será el prime...,2023-03-24,https://www.portafolio.co/negocios/empresas/ec...,EMPRESAS,Ocupará el cargo desde el 1 de abril y hasta q...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",ecopetrol,Portafolio,PORTAFOLIO,"La junta directiva de Ecopetrol, durante una s...",https://www.portafolio.co/negocios/empresas/mo...


In [ ]:
driver.close()